In [1]:
import cv2
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np

# Зчитуємо зображення

In [2]:
img_rafael = cv2.imread('1.jpg')
img_dragon = cv2.imread('2.jpg')
img_double = cv2.imread('double.jpg')

img_warp_rafael = cv2.imread('3.jpg')
img_warp_dragon = cv2.imread('4.jpg')

hT_rafael, wT_rafael, cT_rafael = img_rafael.shape
hT_dragon, wT_dragon, cT_dragon = img_dragon.shape
hT_double,wT_double,cT_double = img_double.shape

# Обчислюємо ключові точки

In [3]:
sift = cv2.SIFT_create(nfeatures=1000)

kp_rafael, des_rafael = sift.detectAndCompute(img_rafael, None)
kp_dragon, des_dragon = sift.detectAndCompute(img_dragon, None)
kp_double, des_double = sift.detectAndCompute(img_double, None)

# Створюємо матчер

In [4]:
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

# Завантажуємо модель

In [5]:
model = keras.models.load_model('./new_model_v_02')

In [12]:
# вебка
webcam = cv2.VideoCapture(0)
# параметри для анімації
counter = 1
x_img = 150
y_img = 150

# розмір кадру
frame_width = webcam.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_height = webcam.get(cv2.CAP_PROP_FRAME_HEIGHT)

# Починаємо зчитувати кадри з вебки і записуємо їх в відео

In [13]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_without = cv2.VideoWriter('out_without_classficator.avi', fourcc = fourcc, fps=30, frameSize=(int(frame_width),int(frame_height)))
out_with = cv2.VideoWriter('outpy_with_classficator.avi', fourcc, 30, (int(frame_width),int(frame_height)))

while True:

    sucess, frame_webcam = webcam.read()
    counter += 1
    if sucess == False:
        print('Error sucess')
        break

    frame_aug = frame_webcam.copy()
    # вираховуємо ключові точки
    kp_webcam, des_webcam = sift.detectAndCompute(frame_webcam, None)

    try:
        # шукаємо спвіпадіння
        matches_rafael = flann.knnMatch(des_rafael, des_webcam,k=2)
        matches_dragon = flann.knnMatch(des_dragon, des_webcam, k=2)
        matches_double = flann.knnMatch(des_double, des_webcam, k=2)
    except:
        cv2.imshow('Test', frame_webcam)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    # відбираємо точки
    good_rafael =[]
    good_dragon = []
    good_double = []

    for m,n in matches_rafael:
        if m.distance < 0.75*n.distance:
            good_rafael.append(m)

    for m,n in matches_dragon:
        if m.distance < 0.75*n.distance:
            good_dragon.append(m)

    for m,n in matches_double:
        if m.distance < 0.75*n.distance:
            good_double.append(m)

    try:
        # класифікуємо
        srcPts = np.float32([kp_double[m.queryIdx].pt for m in good_double]).reshape(-1, 1, 2)
        dstPts = np.float32([kp_webcam[m.trainIdx].pt for m in good_double]).reshape(-1, 1, 2)
        matrix, mask = cv2.findHomography(dstPts,srcPts,cv2.RANSAC,5)

        imgWarp = cv2.warpPerspective(frame_webcam, matrix, (img_double.shape[1], img_double.shape[0]))
        img_warp = cv2.resize(imgWarp, (64, 64))

        t = np.argmax(model.predict(img_warp[np.newaxis]))
    except:
        cv2.imshow('Test', frame_webcam)
        out_without.write(frame_webcam)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        continue

    # рафаель
    if t == 1:
        try:
            srcPts = np.float32([kp_rafael[m.queryIdx].pt for m in good_rafael]).reshape(-1, 1, 2)
            dstPts = np.float32([kp_webcam[m.trainIdx].pt for m in good_rafael]).reshape(-1, 1, 2)
            matrix, mask = cv2.findHomography(srcPts,dstPts,cv2.RANSAC,5)

            # готовимо холст
            pts = np.float32([[0,0],[0,hT_rafael],[wT_rafael,hT_rafael],[wT_rafael,0]]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,matrix)

            # готуємо нове зображення для заповнення
            imgWarp = cv2.warpPerspective(img_warp_rafael, matrix, (frame_webcam.shape[1],frame_webcam.shape[0]))

            maskNew = np.zeros((frame_webcam.shape[0],frame_webcam.shape[1]),np.uint8)
            cv2.fillPoly(maskNew,[np.int32(dst)],(255,255,255))
            maskInv = cv2.bitwise_not(maskNew)
            frame_aug = cv2.bitwise_and(frame_aug, frame_aug, mask = maskInv)
            frame_aug = cv2.bitwise_or(imgWarp,frame_aug)

            # демонструємо перетворене зображення
            cv2.imshow('Test', frame_aug)
            out_with.write(frame_aug)
            out_without.write(frame_webcam)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        except:
            cv2.imshow('Test', frame_webcam)
            out_without.write(frame_webcam)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue
    # дракон
    elif t == 2:
        try:
            srcPts = np.float32([kp_dragon[m.queryIdx].pt for m in good_dragon]).reshape(-1, 1, 2)
            dstPts = np.float32([kp_webcam[m.trainIdx].pt for m in good_dragon]).reshape(-1, 1, 2)
            matrix, mask = cv2.findHomography(srcPts,dstPts,cv2.RANSAC,5)

            # готовимо холст
            pts = np.float32([[0,0],[0,hT_dragon],[wT_dragon,hT_dragon],[wT_dragon,0]]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,matrix)

            # готуємо нове зображення для заповнення
            imgWarp = cv2.warpPerspective(img_warp_dragon, matrix, (frame_webcam.shape[1],frame_webcam.shape[0]))

            maskNew = np.zeros((frame_webcam.shape[0],frame_webcam.shape[1]),np.uint8)
            cv2.fillPoly(maskNew,[np.int32(dst)],(255,255,255))
            maskInv = cv2.bitwise_not(maskNew)
            frame_aug = cv2.bitwise_and(frame_aug, frame_aug, mask = maskInv)
            frame_aug = cv2.bitwise_or(imgWarp,frame_aug)

            # демонструємо перетворене зображення
            cv2.imshow('Test', frame_aug)
            out_with.write(frame_aug)
            out_without.write(frame_webcam)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        except:
            cv2.imshow('Test', frame_webcam)
            out_without.write(frame_webcam)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue
    # перетин зображень
    elif t == 3:
        try:
            if counter <= 16:
                img_name = f'gif/frame-{counter}.gif'
                img = cv2.VideoCapture(img_name)
                _, img = img.read()
                img_height, img_width, _ = img.shape
                frame_aug[ y_img:y_img+img_height , x_img:x_img+img_width ] = img
            else:
                counter = 1

            # демонструємо перетворене зображення
            cv2.imshow('Test', frame_aug)
            out_with.write(frame_aug)
            out_without.write(frame_webcam)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        except:
            cv2.imshow('Test', frame_webcam)
            out_without.write(frame_webcam)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue
    else:
        cv2.imshow('Test', frame_webcam)
        out_with.write(frame_aug)
        out_without.write(frame_webcam)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


In [14]:
webcam.release()
out_without.release()
out_with.release()
cv2.destroyAllWindows()